In [2]:
# import libraries
#from sodapy import Socrata
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta, date
import os
import time
import pyarrow

### Get COVID-19 nursing home data with CMS open API. The variable, latest_uuid will need to be updated according to uuid version updates per https://data.cms.gov/covid-19/covid-19-nursing-home-data/api-docs

In [3]:
# API latest version uuid
latest_uuid = '84e99b2f-e2b5-4986-9bfb-c225fde9a222'

In [4]:
# test data fetch. If response is 200 then succeeded
url1 = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]=2021-06-27&offset=0&size=5000"
response1 = requests.get(url1)
response1


<Response [200]>

In [5]:
# gets all data for a single week up to offset of 15500

nh_results_m1 = []
size=5000

for i in range(0,15500,5000):
    print("----")
    offset = i
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]=2021-06-27&offset={offset}&size={size}"
    print("requesting", url)

    response = requests.get(url)
    nh_data = response.json()
    nh_results_m1.extend(nh_data)
  

----
requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2021-06-27&offset=0&size=5000
----
requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2021-06-27&offset=5000&size=5000
----
requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2021-06-27&offset=10000&size=5000
----
requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2021-06-27&offset=15000&size=5000


In [6]:
len(nh_results_m1)

15250

In [7]:
nh_results_m1[-1]

{'week_ending': '2021-06-27',
 'federal_provider_number': '686124',
 'provider_name': 'HARMONY HEALTH CENTER',
 'provider_address': '9820 N KENDALL DRIVE',
 'provider_city': 'MIAMI',
 'provider_state': 'FL',
 'provider_zip_code': '33176',
 'provider_phone_number': '3052716311',
 'county': 'Miami-Dade',
 'submitted_data': 'Y',
 'passed_quality_assurance_check': 'Y',
 'residents_weekly_confirmed_covid_19': '0',
 'residents_total_confirmed_covid_19': '39',
 'residents_weekly_all_deaths': '1',
 'residents_total_all_deaths': '46',
 'residents_weekly_covid_19_deaths': '0',
 'residents_total_covid_19_deaths': '2',
 'number_of_all_beds': '203',
 'total_number_of_occupied_beds': '192',
 'Residents_Hospitalizations_with_Confirmed_COVID_19': '0',
 'Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines': '0',
 'staff_weekly_confirmed_covid_19': '0',
 'staff_total_confirmed_covid_19': '61',
 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents': '0',
 'weekly_resi

### Each week returns approx 15250 rows, sometimes more, but does not exceed 15500. I have set up a nested loop (whole/for) to accomodate offset and date increments. It retrieves eight weeks of data, which is saved in a dataframe. For 8 wks, num of rows = approx 122000.
### To get a year's worth of data, I re-ran for each 8(ish) week chunk between 6-27-21 and 7-10-22.

### Each 8-wk dataframe is then subset to keep only needed cols before concatenation. 

### Excluding rows for which "passed data quality assurance check" = 'N'

In [40]:
# First batch: week ending 6-27-21 to week ending 8-15-21

nh_all_Jun27_21_Aug15_21 = []
week_ending_start = date(2021,6,27)
week_ending_end = date(2021,8,15)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Jun27_21_Aug15_21.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-06-27&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-06-27&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-06-27&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-06-27&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-06-27&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-07-04&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [80]:
nh_all_Jun27_21_Aug15_21 = pd.DataFrame(nh_all_Jun27_21_Aug15_21)
nh_all_Jun27_21_Aug15_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2021-06-27,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,,,0,,,0,0,0
1,2021-06-27,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,,,0,,,0,0,0
2,2021-06-27,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,,,0,,,0,0,0
3,2021-06-27,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,0,0,,,0,,,0,0,0
4,2021-06-27,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,,,0,,,0,0,0


In [81]:
nh_all_Jun27_21_Aug15_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
121979,2021-08-15,676499,SPRINGTOWN PARK REHABILITATION AND CARE CENTER,201 WILLIAMS WARD RD.,SPRINGTOWN,TX,76082,5124382630,Parker,Y,...,0,0,,,0,,,0,0,0
121980,2021-08-15,686123,KENDALL LAKES HEALTHCARE AND REHAB CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,7864337400,Miami-Dade,Y,...,0,0,,,0,,,0,0,0
121981,2021-08-15,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,0,0,,,0,,,0,0,0
121982,2021-08-15,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,N,...,,,,,,,,,,
121983,2021-08-15,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,0,0,,,0,,,0,0,0


In [82]:
nh_all_Jun27_21_Aug15_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121984 entries, 0 to 121983
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 112.6+ MB


In [44]:
# Second batch: 8-22-21 to 10-10-21

nh_all_Aug22_21_Oct10_21 = []
week_ending_start = date(2021,8,22)
week_ending_end = date(2021,10,10)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Aug22_21_Oct10_21.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-08-22&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-08-22&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-08-22&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-08-22&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-08-22&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-08-29&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [83]:
nh_all_Aug22_21_Oct10_21 = pd.DataFrame(nh_all_Aug22_21_Oct10_21)
nh_all_Aug22_21_Oct10_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2021-08-22,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,,,0,,,0,0,0
1,2021-08-22,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,,,0,,,0,0,0
2,2021-08-22,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,,,0,,,0,0,0
3,2021-08-22,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,0,0,,,0,,,0,0,0
4,2021-08-22,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,,,0,,,0,0,0


In [46]:
nh_all_Aug22_21_Oct10_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
121921,2021-10-10,686123,KENDALL LAKES HEALTHCARE AND REHAB CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,7864337400,Miami-Dade,Y,...,0,0,62,,31,,,0,0,0
121922,2021-10-10,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,0,0,0,,0,,,0,0,0
121923,2021-10-10,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,...,0,0,0,,0,,,0,0,0
121924,2021-10-10,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,0,0,1,,2.222,,,0,0,0
121925,2021-10-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,0,0,0,,0,,,0,0,0


In [84]:
nh_all_Aug22_21_Oct10_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121926 entries, 0 to 121925
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 112.6+ MB


In [48]:
# Third batch: 10-17-21 to 12-05-21

nh_all_Oct17_21_Dec05_21 = []
week_ending_start = date(2021,10,17)
week_ending_end = date(2021,12,5)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Oct17_21_Dec05_21.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-10-17&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-10-17&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-10-17&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-10-17&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-10-17&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-10-24&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [85]:
nh_all_Oct17_21_Dec05_21 = pd.DataFrame(nh_all_Oct17_21_Dec05_21)
nh_all_Oct17_21_Dec05_21.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2021-10-17,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,0,,0,,,0,0,0
1,2021-10-17,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,2,,4.348,,,0,0,0
2,2021-10-17,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,16,,26.23,,,0,0,0
3,2021-10-17,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,0,0,29,,43.284,,,0,0,0
4,2021-10-17,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,0,,0,,,0,0,0


In [86]:
nh_all_Oct17_21_Dec05_21.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
121818,2021-12-05,686124,HARMONY HEALTH CENTER,9820 N KENDALL DRIVE,MIAMI,FL,33176,3052716311,Miami-Dade,Y,...,0,0,36,,23.529,,,0,0,0
121819,2021-12-05,745000,LAS ALTURAS DE PENITAS,414 LIBERTY BLVD.,PENITAS,TX,78576,9569970800,Hidalgo,Y,...,0,0,4,,6.78,,,0,0,0
121820,2021-12-05,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,0,0,13,,26.531,,,0,0,0
121821,2021-12-05,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,0,0,1,,20,,,0,0,0
121822,2021-12-05,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,Y,...,0,0,0,,0,,,0,0,0


In [87]:
nh_all_Oct17_21_Dec05_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121823 entries, 0 to 121822
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 112.5+ MB


In [21]:
# Fourth batch: 12-12-21 to 1-30-22

nh_all_Dec12_21_Jan30_22 = []
week_ending_start = date(2021,12,12)
week_ending_end = date(2022,1,30)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Dec12_21_Jan30_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-12-12&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-12-12&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-12-12&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-12-12&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-12-12&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2021-12-19&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [88]:
nh_all_Dec12_21_Jan30_22 = pd.DataFrame(nh_all_Dec12_21_Jan30_22)
nh_all_Dec12_21_Jan30_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2021-12-12,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,34,,56.667,,,0,0,0
1,2021-12-12,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,4,,7.692,,,0,0,0
2,2021-12-12,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,27,,45,,,0,0,0
3,2021-12-12,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,0,0,58,,64.444,,,0,0,0
4,2021-12-12,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,37,,50.685,,,0,0,0


In [89]:
nh_all_Dec12_21_Jan30_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
121648,2022-01-30,745001,ESTATES AT SHAVANO PARK,4366 LOCKHILL SELMA,SHAVANO PARK,TX,78249,5124382630,Bexar,Y,...,0,0,33,,57.895,,,0,0,0
121649,2022-01-30,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,Y,...,0,0,6,,24,,,0,0,0
121650,2022-01-30,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,0,0,2,,33.333,,,0,0,0
121651,2022-01-30,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,N,...,,,,,,,,,,
121652,2022-01-30,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,Y,...,0,0,0,,0,,,0,0,0


In [90]:
nh_all_Dec12_21_Jan30_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121653 entries, 0 to 121652
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 112.3+ MB


In [57]:
# Fifth batch: 2-6-22 to 3-27-22

nh_all_Feb06_22_Mar27_22 = []
week_ending_start = date(2022,2,6)
week_ending_end = date(2022,3,27)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Feb06_22_Mar27_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-02-06&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-02-06&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-02-06&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-02-06&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-02-06&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-02-13&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [91]:
nh_all_Feb06_22_Mar27_22 = pd.DataFrame(nh_all_Feb06_22_Mar27_22)
nh_all_Feb06_22_Mar27_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-02-06,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,28,,40,,,0,0,0
1,2022-02-06,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,4,,5.97,,,0,0,0
2,2022-02-06,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,40,,52.632,,,0,0,0
3,2022-02-06,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,0,0,46,,50,,,0,0,0
4,2022-02-06,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,44,,47.826,,,0,0,0


In [92]:
nh_all_Feb06_22_Mar27_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
121517,2022-03-27,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,Y,...,0,0,16,,72.727,,,0,0,0
121518,2022-03-27,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,0,0,2,,50,,,0,0,0
121519,2022-03-27,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,N,...,,,,,,,,,,
121520,2022-03-27,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,N,...,,,,,,,,,,
121521,2022-03-27,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,Y,...,0,0,14,,29.167,,,0,0,0


In [93]:
nh_all_Feb06_22_Mar27_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121522 entries, 0 to 121521
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 112.2+ MB


In [61]:
# Sixth batch: 4-3-22 to 5-22-22

nh_all_Apr03_22_May22_22 = []
week_ending_start = date(2022,4,3)
week_ending_end = date(2022,5,22)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Apr03_22_May22_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-04-03&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-04-03&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-04-03&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-04-03&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-04-03&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-04-10&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [94]:
nh_all_Apr03_22_May22_22=pd.DataFrame(nh_all_Apr03_22_May22_22)
nh_all_Apr03_22_May22_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-04-03,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,28,,37.838,,,0,0,0
1,2022-04-03,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,16,,11.429,,,0,0,0
2,2022-04-03,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,50,,66.667,,,0,0,0
3,2022-04-03,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,0,0,55,,63.218,,,0,0,0
4,2022-04-03,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,52,,44.444,,,0,0,0


In [95]:
nh_all_Apr03_22_May22_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
121402,2022-05-22,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,Y,...,0,0,9,,37.5,,,0,0,0
121403,2022-05-22,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,0,0,1,,10,,,0,0,0
121404,2022-05-22,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,N,...,,,,,,,,,,
121405,2022-05-22,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,Y,...,0,0,54,,80.597,,,0,0,0
121406,2022-05-22,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,Y,...,0,0,17,,28.814,,,0,0,0


In [96]:
nh_all_Apr03_22_May22_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121407 entries, 0 to 121406
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 112.1+ MB


In [65]:
# Seventh batch: 5-29-22 to 7-10-22

nh_all_May29_22_Jul10_22 = []
week_ending_start = date(2022,5,29)
week_ending_end = date(2022,7,10)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_May29_22_Jul10_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-05-29&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-05-29&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-05-29&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-05-29&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-05-29&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-06-05&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [97]:
nh_all_May29_22_Jul10_22=pd.DataFrame(nh_all_May29_22_Jul10_22)
nh_all_May29_22_Jul10_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-05-29,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,0,0,28,,42.424,,,0,0,0
1,2022-05-29,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,0,0,17,,11.565,,,0,0,0
2,2022-05-29,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,0,0,54,,71.053,,,0,0,0
3,2022-05-29,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,N,...,,,,,,,,,,
4,2022-05-29,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,0,0,60,,63.83,,,0,0,0


In [98]:
nh_all_May29_22_Jul10_22.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
106104,2022-07-10,745002,KENT COUNTY NURSING HOME,1443 NORTH MAIN,JAYTON,TX,79528,8062373036,Kent,Y,...,0,0,10,,32.258,3,,5.556,5.66,9.677
106105,2022-07-10,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,...,0,0,1,,10,0,,0,0,0
106106,2022-07-10,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,N,...,,,,,,,,,,
106107,2022-07-10,745005,CENTER AT ZARAGOZA,12660 PEBBLE HILLS BLVD.,EL PASO,TX,79938,9159901700,El Paso,Y,...,0,0,56,,83.582,35,,50.725,50.725,52.239
106108,2022-07-10,745006,FIVE POINTS NURSING AND REHABILITATION,1901 N. HAMPTON RD.,DESOTO,TX,75115,9726949810,Dallas,Y,...,0,0,22,,31.429,0,,0,0,0


In [99]:
nh_all_May29_22_Jul10_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106109 entries, 0 to 106108
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 98.0+ MB


In [69]:
# Eighth batch: 7-17-22 to 9-25-22

nh_all_Jul17_22_Sept25_22 = []
week_ending_start = date(2022,7,17)
week_ending_end = date(2022,9,25)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Jul17_22_Sept25_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-07-17&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-07-17&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-07-17&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-07-17&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-07-17&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-07-24&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [100]:
nh_all_Jul17_22_Sept25_22=pd.DataFrame(nh_all_Jul17_22_Sept25_22)
nh_all_Jul17_22_Sept25_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-07-17,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,62.264,64.706,26,,37.143,24,,29.63,30,34.286
1,2022-07-17,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,57.692,66.176,17,,11.486,0,,0,0,0
2,2022-07-17,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,91.667,91.667,61,,81.333,59,,74.684,74.684,78.667
3,2022-07-17,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,93.59,97.333,74,,80.435,72,,75.789,75.789,78.261
4,2022-07-17,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,57.303,60.714,70,,60.87,56,,40.58,40.58,48.696


In [71]:
# Ninth batch: 10-02-22 to 11-27-22

nh_all_Oct02_22_Nov27_22 = []
week_ending_start = date(2022,10,2)
week_ending_end = date(2022,11,27)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_all_Oct02_22_Nov27_22.extend(data)
    
        offset = offset + 5000
        time.sleep(5)

    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-10-02&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-10-02&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-10-02&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-10-02&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-10-02&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data?filter[week_ending]=2022-10-09&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a3371812-a39f-4804-8105-7fc4bc19d57f/data

In [101]:
nh_all_Oct02_22_Nov27_22=pd.DataFrame(nh_all_Oct02_22_Nov27_22)
nh_all_Oct02_22_Nov27_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-10-02,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,82.456,88.679,24,,34.286,24,,29.63,30,34.286
1,2022-10-02,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,71.233,78.788,18,,12.245,0,,0,0,0
2,2022-10-02,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,47.222,47.222,57,,73.077,1,,1.176,1.176,1.282
3,2022-10-02,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,96.203,97.436,76,,80,86,,88.66,88.66,90.526
4,2022-10-02,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,24.444,25.882,72,,63.717,11,,8.148,8.148,9.735


### concat dfs

In [ ]:
#all
nh_Jun27_21_Nov27_22_concat = pd.concat([nh_all_Jun27_21_Aug15_21,
                                             nh_all_Aug22_21_Oct10_21,
                                             nh_all_Oct17_21_Dec05_21,
                                             nh_all_Dec12_21_Jan30_22,
                                             nh_all_Feb06_22_Mar27_22,
                                             nh_all_Apr03_22_May22_22,
                                             nh_all_May29_22_Jul10_22,
                                             nh_all_Jul17_22_Sept25_22,
                                             nh_all_Oct02_22_Nov27_22]).reset_index(drop=True)

### save to parquet files

In [104]:
nh_all_Jun27_21_Nov27_22_concat.to_parquet('data/nh_all_Jun27_21_Nov27_22_concat.parquet.gzip',
                                          compression='gzip')

In [5]:
nh_latest = []

#Data is aggregated to week. Week is Monday - Sunday,

#Enter the Sunday (week ending date) of the week you want to start on.
week_ending_start = date(2022,12,25)

#Enter the Sunday (week ending date) of the week you want to end on.
week_ending_end = date(2023,1,1)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_latest.extend(data)
    
        offset = offset + size
        time.sleep(5)

    #when viewing output, ensure that offset increments by the size amount for each week
    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data?filter[week_ending]=2022-12-25&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data?filter[week_ending]=2022-12-25&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data?filter[week_ending]=2022-12-25&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data?filter[week_ending]=2022-12-25&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data?filter[week_ending]=2022-12-25&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data?filter[week_ending]=2023-01-01&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/a9e94001-fa39-47d5-9ad7-c615e5399756/data

In [15]:
nh_latest = pd.DataFrame(nh_latest)
nh_latest.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Residents_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines_with_No_Medical_Contraindications,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,88.679,95.918,24,,34.286,24,,29.63,30,34.286
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,71.951,78.667,22,,14.765,1,,0.541,0.546,0.671
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,92.105,92.105,56,,69.136,15,,17.442,17.442,18.519
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,96,98.63,82,,88.172,30,,30.928,30.928,32.258
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,70.652,76.471,72,,62.609,5,,3.597,3.597,4.348


In [18]:
#concat new with old, save to a new df with date as last week start date
nh_all = pd.concat([nh_latest, nh_old]).reset_index(drop=True)

In [19]:
#testing for same number of cols between old and latest dfs
if len(nh_all) == len(nh_old) + len(nh_latest):
    print('proceed')
else:
    print('check cols')

proceed


### Updating data - CMS has pared down the number of fields as of summer 2023. Pulling for year of 2023, I will keep that data in a separate file

In [16]:
#Update file name to read in currently existing parquet file and save to df
nh_62721_31923=pd.read_parquet('data/data_pre_proc/nh_all_Jun27_21_Mar19_23_concat.parquet.gzip')


In [8]:
#Update to pull latest data from cms api (add 7 days to the last week ending date from the currently existing data, i.e.
# if currently existing ending date is 3-5-23, then the new start week ending date will be 3-12-23. Remember weeks run Mon-Sun, 
#so week ending date will always be on a Sunday)

nh_1823_8623 = []

#Data is aggregated to week level.

#Enter the Sunday (week ending date) of the week you want to start on.
week_ending_start = date(2023,1,8)

#Enter the Sunday (week ending date) of the week you want to end on.
week_ending_end = date(2023,8,6)
week = timedelta(days=7)
offset = 0
size = 5000

while week_ending_start <= week_ending_end:
    
    for offset in range(0,20000,5000):
        print("---")
        url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
        print("Requesting",url)
        
        response = requests.get(url)
        data = response.json()
        if len(data) == 0:
            break
        nh_1823_8623.extend(data)
    
        offset = offset + size
        time.sleep(5)

    #when viewing output, ensure that offset increments by the size amount for each week
    print("---")
    url = f"https://data.cms.gov/data-api/v1/dataset/{latest_uuid}/data?filter[week_ending]={week_ending_start}&offset={offset}&size={size}"
    print("Requesting",url)
    
    
    week_ending_start = week_ending_start + week
    time.sleep(3)

---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-01-08&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-01-08&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-01-08&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-01-08&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-01-08&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-01-15&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data

---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-03-26&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-03-26&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-03-26&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-03-26&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-03-26&offset=20000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-04-02&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data

---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-06-11&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-06-11&offset=5000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-06-11&offset=10000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-06-11&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-06-11&offset=15000&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data?filter[week_ending]=2023-06-18&offset=0&size=5000
---
Requesting https://data.cms.gov/data-api/v1/dataset/84e99b2f-e2b5-4986-9bfb-c225fde9a222/data

In [9]:
#save api result to a dataframe
nh_1823_8623 = pd.DataFrame(nh_1823_8623)
nh_1823_8623.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,...,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
0,2023-01-08,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,...,81,70,,86.42,47,,88.679,24,,29.63
1,2023-01-08,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,...,178,144,,80.899,55,,73.333,1,,0.562
2,2023-01-08,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,...,83,77,,92.771,35,,89.744,15,,18.072
3,2023-01-08,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,...,96,93,,96.875,72,,94.737,30,,31.25
4,2023-01-08,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,...,142,117,,82.394,65,,72.222,6,,4.225


In [17]:
#concat new with the df of existing data (if applicable), save to a new df with date as latest week ending date
nh_62721_31923 = pd.concat([nh_31223_31923,nh_62721_3523]).reset_index(drop=True)

In [18]:
#testing for same number of cols between old and latest dfs, if applicable
if len(nh_62721_31923) == len(nh_62721_3523) + len(nh_31223_31923):
    print('proceed')
else:
    print('check cols')

proceed


### export data to data pre proc folder

In [11]:
#save concated as updated parquet file
nh_1823_8623.to_parquet('data_pre_proc/nh_all_Jan8_23_Aug6_23_concat.parquet.gzip',
                                          compression='gzip')

#### Thanks to: Jonathan Soma's API tutorial: https://www.youtube.com/watch?v=4Fdyft-ky0w and Jennifer Peebles' documentation on working with the CMS COVID-19 Nursing Home data API: https://github.com/jenniferpeebles/covid_in_nursing_homes_data_from_cms
